In [ ]:
import customtkinter as ctk
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image, ImageDraw

ctk.set_appearance_mode("dark")

class DigitCanvasApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("NEURAL VISION v3.0 - Draw & Predict")
        self.geometry("500x700")
        self.configure(fg_color="#0B0F19")

        # Charger le modèle
        try:
            self.model = tf.keras.models.load_model('handwritten_model_v2.keras')
        except:
            print("Erreur: Modèle introuvable. Vérifiez le fichier .keras")

        # Titre
        self.label = ctk.CTkLabel(self, text="DESSINEZ UN CHIFFRE", font=("Segoe UI", 24, "bold"), text_color="#00F2FF")
        self.label.pack(pady=20)

        # Zone de dessin (Canvas)
        self.canvas = ctk.CTkCanvas(self, width=300, height=300, bg="black", highlightthickness=0)
        self.canvas.pack(pady=10)
        self.canvas.bind("<B1-Motion>", self.draw)

        # Image interne pour PIL (pour l'IA)
        self.pil_img = Image.new("L", (300, 300), 0)
        self.draw_obj = ImageDraw.Draw(self.pil_img)

        # Affichage du résultat
        self.result_label = ctk.CTkLabel(self, text="-", font=("Segoe UI", 60, "bold"), text_color="white")
        self.result_label.pack(pady=10)

        # Boutons
        self.btn_predict = ctk.CTkButton(self, text="PRÉDIRE", fg_color="#00F2FF", text_color="black", 
                                        font=("Segoe UI", 14, "bold"), command=self.predict)
        self.btn_predict.pack(pady=10)

        self.btn_clear = ctk.CTkButton(self, text="EFFACER", fg_color="#1E2638", command=self.clear_canvas)
        self.btn_clear.pack(pady=10)

    def draw(self, event):
        x, y = event.x, event.y
        r = 10 # Épaisseur du trait
        self.canvas.create_oval(x-r, y-r, x+r, y+r, fill="white", outline="white")
        self.draw_obj.ellipse([x-r, y-r, x+r, y+r], fill=255)

    def clear_canvas(self):
        self.canvas.delete("all")
        self.pil_img = Image.new("L", (300, 300), 0)
        self.draw_obj = ImageDraw.Draw(self.pil_img)
        self.result_label.configure(text="-")

    def predict(self):
        # 1. Convertir en array numpy
        img_array = np.array(self.pil_img)

        # 2. Centrer le chiffre (très important pour éviter le "toujours 1")
        # On cherche les contours du dessin
        coords = cv2.findNonZero(img_array)
        if coords is not None:
            x, y, w, h = cv2.boundingRect(coords)
            # On découpe le chiffre
            digit_crop = img_array[y:y+h, x:x+w]
            # On ajoute une marge (padding) de 4 pixels pour ne pas toucher les bords
            digit_padded = cv2.copyMakeBorder(digit_crop, 4, 4, 4, 4, cv2.BORDER_CONSTANT, value=0)
        else:
            return

        # 3. Redimensionner à 28x28
        img_28 = cv2.resize(digit_padded, (28, 28), interpolation=cv2.INTER_AREA)

        # 4. Normaliser pour le CNN
        img_final = img_28.reshape(1, 28, 28, 1).astype('float32') / 255.0

        # 5. Prédiction
        pred = self.model.predict(img_final)
        digit = np.argmax(pred)
        
        self.result_label.configure(text=str(digit))

if __name__ == "__main__":
    app = DigitCanvasApp()
    app.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
